# Palmer Penguins Dataset Analysis
## CS14003 - Project 2: Decision Tree

**Dataset**: Palmer Penguins Dataset  
**Task**: Multi-class classification (Species: Adelie, Chinstrap, Gentoo)  
**Samples**: 344  
**Features**: Physical characteristics (bill, flipper, body mass, etc.)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Import custom utilities
import sys
sys.path.append('../')
from utils.preprocess import *
from utils.train_model import *
from utils.visualize import *
# Set random seed for reproducibility
np.random.seed(42)

print("="*60)
print("PALMER PENGUINS DATASET ANALYSIS")
print("="*60)


## 1. Data Loading and Exploration